In [1]:
import matplotlib.pyplot as plt
import anndata
import scanpy as sc
import snapatac2 as snap
import numpy as np
import pandas as pd
import os
import scanpy.external as sce
import seaborn as sns
from sklearn.metrics import silhouette_score


In [2]:
import warnings
warnings.filterwarnings("ignore")


In [3]:
adata_concat = snap.read_dataset('/data2st1/junyi/output/atac0627/doublet_filtered.h5ads/_dataset.h5ads')

In [4]:
df_meta_all = pd.read_csv('/data2st1/junyi/output/atac0627/ATACSC_3REGION_ALL_L2annoated.csv',index_col=0)

In [5]:
df_l3l4 = pd.read_csv('/data2st1/junyi/output/atac0627/iterative/annotated_l3l4.csv',index_col=0)

In [6]:
df_meta_all.head()

,sample,doublet_probability,doublet_score,leiden,leiden_default,leiden_res_0.1,leiden_res_0.2,leiden_res_0.3,leiden_res_0.4,leiden_res_0.5,...,leiden_res_1.9,celltype.L1,celltype.L2,Neurotransmitter_celltype,celltype.L1_ct,Sample_name,Condition,Region,celltype.L2.raw,region_nt
MC37A_AMY:AAACGAAAGAGTGGAA-1,MC37A_AMY,0.092856,0.130307,2,6,0,0,1,1,5,...,2,Neuron,AMY Strip2_Penk GABA,GABAergic,Neuron,MC37A_AMY,MC,AMY,AMY Strip2_Penk GABA,AMY_GABA
MC37A_AMY:AAACGAAAGGGTAGTC-1,MC37A_AMY,0.128610,0.014116,18,7,4,5,5,6,7,...,18,Immune,Microglia-3,NN,Immune,MC37A_AMY,MC,AMY,Microglia-3,NN
MC37A_AMY:AAACGAAGTACGGAGT-1,MC37A_AMY,0.118241,0.019370,4,1,0,0,1,1,1,...,4,Neuron,AMY Lhx6_Maf GABA,GABAergic,Neuron,MC37A_AMY,MC,AMY,AMY Lhx6_Maf GABA,AMY_GABA
MC37A_AMY:AAACGAAGTCAGCAAG-1,MC37A_AMY,0.074653,0.063387,6,6,0,0,1,1,5,...,6,Neuron,AMY Strip2_Penk GABA,GABAergic,Neuron,MC37A_AMY,MC,AMY,AMY Strip2_Penk GABA,AMY_GABA
MC37A_AMY:AAACGAAGTCCGAGCT-1,MC37A_AMY,0.098286,0.032532,1,8,0,0,1,1,1,...,1,Neuron,AMY Hgf_Satb2 Glut,Glutamatergic,Neuron,MC37A_AMY,MC,AMY,AMY Hgf_Satb2 Glut,AMY_Glut


In [7]:
df_l3l4.head()

,celltype.L3,celltype.L4,celltype.L4.raw
MC50B_PFC:AAGATAGCAGGTAACG-1,PFC L6b Glut-0,PFC L6b Glut-0-0,NaN
MC50B_PFC:ACTAGGTAGTGTGTAA-1,PFC L6b Glut-1,PFC L6b Glut-1-0,NaN
MC50B_PFC:AGGCGTCCAATTCTCT-1,PFC L6b Glut-0,PFC L6b Glut-0-0,NaN
MC50B_PFC:AGGCGTCTCCATGTTT-1,PFC L6b Glut-1,PFC L6b Glut-1-0,NaN
MC50B_PFC:CCCGTTATCCTCCTGA-1,PFC L6b Glut-1,PFC L6b Glut-1-0,NaN


In [8]:
df_meta_all = df_meta_all.merge(df_l3l4[['celltype.L3','celltype.L4']],left_index=True,right_index=True,how='left')

In [9]:
df_meta_all['celltype.L3'] = df_meta_all['celltype.L3'].fillna(df_meta_all['celltype.L2']+"-0").str.replace("/","-").str.replace(" ","_")
df_meta_all['celltype.L4'] = df_meta_all['celltype.L4'].fillna(df_meta_all['celltype.L3']+"-0").str.replace("/","-").str.replace(" ","_")
df_meta_all['celltype.L2'] = df_meta_all['celltype.L2'].str.replace("/","-").str.replace(" ","_")
df_meta_all['Neurotransmitter_celltype'] = df_meta_all['Neurotransmitter_celltype'].fillna("NN")
df_meta_all['region_nt'] = df_meta_all['region_nt'].fillna("NN")


In [10]:
df_meta_all.groupby(['celltype.L3']).size().reset_index(name='count').to_csv("/data2st1/junyi/output/atac0627/iterative/L3_count_merged.csv",index=False)

In [11]:
df_meta_all.groupby(['celltype.L2']).size().reset_index(name='count').to_csv("/data2st1/junyi/output/atac0627/iterative/L2_count_merged.csv",index=False)

In [12]:
df_meta_all.groupby(['celltype.L4']).size().reset_index(name='count').to_csv("/data2st1/junyi/output/atac0627/iterative/L4_count_merged.csv",index=False)

In [13]:
assert (adata_concat.obs_names == df_meta_all.index).all()
for col in df_meta_all.columns:
    adata_concat.obs[col] = df_meta_all[col]

In [14]:
adata_concat

AnnDataSet object with n_obs x n_vars = 176318 x 526765 backed at '/data2st1/junyi/output/atac0627/doublet_filtered.h5ads/_dataset.h5ads'
contains 18 AnnData objects with keys: 'MC37A_AMY', 'MC39C_HIP', 'MC48D_HIP', 'MC50B_AMY', 'MC50B_HIP', 'MC50B_PFC', 'MC52E_AMY', 'MC52E_PFC', 'MW45A_HIP', 'MW47A_AMY', 'MW47A_HIP', 'MW47A_PFC', 'MW51A_AMY', 'MW51A_HIP', 'MW51A_PFC', 'MC25A_PFC', 'MW26A_PFC', 'MW65A_AMY'
    obs: 'sample', 'doublet_probability', 'doublet_score', 'leiden', 'leiden_default', 'leiden_res_0.1', 'leiden_res_0.2', 'leiden_res_0.3', 'leiden_res_0.4', 'leiden_res_0.5', 'leiden_res_0.6', 'leiden_res_0.7', 'leiden_res_0.8', 'leiden_res_0.9', 'leiden_res_1.0', 'leiden_res_1.1', 'leiden_res_1.2', 'leiden_res_1.3', 'leiden_res_1.4', 'leiden_res_1.5', 'leiden_res_1.6', 'leiden_res_1.7', 'leiden_res_1.8', 'leiden_res_1.9', 'celltype.L2.Condition', 'celltype.L1', 'celltype.L2', 'Neurotransmitter_celltype', 'celltype.L1_ct', 'Sample_name', 'Condition', 'Region', 'celltype.L2.raw', 'r

In [16]:
%time snap.tl.macs3(adata_concat, groupby='celltype.L4',n_jobs=48)

2025-07-30 11:17:53 - INFO - Exporting fragments...
2025-07-30 11:47:08 - INFO - Calling peaks...
100%|██████████| 238/238 [1:05:55<00:00, 16.62s/it] 


CPU times: user 1h 17min 15s, sys: 1h 5min 25s, total: 2h 22min 41s
Wall time: 1h 37min


In [17]:
%time peaks = snap.tl.merge_peaks(adata_concat.uns['macs3'], snap.genome.mm10)


CPU times: user 3min 12s, sys: 10.3 s, total: 3min 23s
Wall time: 53.5 s


In [21]:
peaks.shape

(1635846, 239)

In [19]:
adata_concat

Closed AnnDataSet object

In [27]:
peaks.to_pandas().to_csv('/data2st1/junyi/output/atac0627/iterative/peaks.csv')

In [ ]:
black_list = ['Immune','OPC-Oligo','Doublet','PFC Doublet','PFC Not sure','Not sure','Astro-Epen']

In [14]:
from snapatac2._snapatac2 import read_motifs, PyDNAMotif

def cis_bp_mouse(unique: bool = True , path="data/motifdb/Mus_musculus.meme") -> list[PyDNAMotif]:
    motifs = read_motifs(path)
    for motif in motifs:
        motif.name = motif.id.split('+')[0]
    if unique:
        unique_motifs = {}
        for motif in motifs:
            name = motif.name
            if (
                    name not in unique_motifs or 
                    unique_motifs[name].info_content() < motif.info_content()
               ):
               unique_motifs[name] = motif
        motifs = list(unique_motifs.values())
    return motifs


In [28]:
%time peak_mat = snap.pp.make_peak_matrix(adata_concat, use_rep=peaks['Peaks'])


CPU times: user 2h 42min 52s, sys: 42min 23s, total: 3h 25min 15s
Wall time: 10min 12s


In [72]:
adata_concat.obsm

AxisArrays (row) with keys: X_spectral, X_umap

In [29]:
peak_mat.obsm['X_umap'] = adata_concat.obsm['X_umap']
peak_mat.obsm['X_spectral'] = adata_concat.obsm['X_spectral']


In [30]:
peak_mat.layers['count'] = peak_mat.X.copy()
sc.pp.normalize_total(peak_mat)
sc.pp.log1p(peak_mat)
peak_mat.obs['expriment']= peak_mat.obs['sample'].str[:2]
peak_mat.write(f"output/atac0627/3REGIONS_peak.h5ads")


KeyboardInterrupt: 

In [43]:
adata_concat = snap.read_dataset('/data2st1/junyi/output/atac0627/doublet_filtered.h5ads/_dataset.h5ads')

In [44]:
adata_concat

AnnDataSet object with n_obs x n_vars = 176318 x 526765 backed at '/data2st1/junyi/output/atac0627/doublet_filtered.h5ads/_dataset.h5ads'
contains 18 AnnData objects with keys: 'MC37A_AMY', 'MC39C_HIP', 'MC48D_HIP', 'MC50B_AMY', 'MC50B_HIP', 'MC50B_PFC', 'MC52E_AMY', 'MC52E_PFC', 'MW45A_HIP', 'MW47A_AMY', 'MW47A_HIP', 'MW47A_PFC', 'MW51A_AMY', 'MW51A_HIP', 'MW51A_PFC', 'MC25A_PFC', 'MW26A_PFC', 'MW65A_AMY'
    obs: 'sample', 'doublet_probability', 'doublet_score', 'leiden', 'leiden_default', 'leiden_res_0.1', 'leiden_res_0.2', 'leiden_res_0.3', 'leiden_res_0.4', 'leiden_res_0.5', 'leiden_res_0.6', 'leiden_res_0.7', 'leiden_res_0.8', 'leiden_res_0.9', 'leiden_res_1.0', 'leiden_res_1.1', 'leiden_res_1.2', 'leiden_res_1.3', 'leiden_res_1.4', 'leiden_res_1.5', 'leiden_res_1.6', 'leiden_res_1.7', 'leiden_res_1.8', 'leiden_res_1.9', 'celltype.L2.Condition', 'celltype.L1', 'celltype.L2', 'Neurotransmitter_celltype', 'celltype.L1_ct', 'Sample_name', 'Condition', 'Region', 'celltype.L2.raw', 'r

In [45]:
%time dmr_mat = snap.pp.make_peak_matrix(adata_concat,peak_file='/data1st2/hannan_25/data/Nanopore_processV1/nanopore_08_differential/summary/dmr_seg_anno_2tools_nofilter.bed')


CPU times: user 6h 29min 24s, sys: 1h 2min 29s, total: 7h 31min 54s
Wall time: 25min 47s


In [47]:
dmr_mat.write(f"output/atac0627/3REGIONS_dmr_new.h5ads")

In [48]:
adata_concat.close()

In [49]:
adata_concat

Closed AnnDataSet object

In [41]:
dmr_mat = sc.read_h5ad(f"output/atac0627/3REGIONS_dmr.h5ads",backed='r')

In [42]:
dmr_mat

AnnData object with n_obs × n_vars = 176318 × 208269 backed at 'output/atac0627/3REGIONS_dmr.h5ads'
    obs: 'sample', 'doublet_probability', 'doublet_score', 'leiden', 'leiden_default', 'leiden_res_0.1', 'leiden_res_0.2', 'leiden_res_0.3', 'leiden_res_0.4', 'leiden_res_0.5', 'leiden_res_0.6', 'leiden_res_0.7', 'leiden_res_0.8', 'leiden_res_0.9', 'leiden_res_1.0', 'leiden_res_1.1', 'leiden_res_1.2', 'leiden_res_1.3', 'leiden_res_1.4', 'leiden_res_1.5', 'leiden_res_1.6', 'leiden_res_1.7', 'leiden_res_1.8', 'leiden_res_1.9', 'celltype.L2.Condition', 'celltype.L1', 'celltype.L2', 'Neurotransmitter_celltype', 'celltype.L1_ct', 'Sample_name', 'Condition', 'Region', 'celltype.L2.raw', 'region_nt', 'celltype.L3', 'celltype.L4'

In [5]:
peak_mat.obs.columns

Index(['sample', 'doublet_probability', 'doublet_score', 'leiden',
       'leiden_default', 'leiden_res_0.1', 'leiden_res_0.2',
       'leiden_res_0.30000000000000004', 'leiden_res_0.4', 'leiden_res_0.5',
       'leiden_res_0.6', 'leiden_res_0.7000000000000001', 'leiden_res_0.8',
       'leiden_res_0.9', 'leiden_res_1.0', 'leiden_res_1.1',
       'leiden_res_1.2000000000000002', 'leiden_res_1.3000000000000003',
       'leiden_res_1.4000000000000001', 'leiden_res_1.5000000000000002',
       'leiden_res_1.6', 'leiden_res_1.7000000000000002',
       'leiden_res_1.8000000000000003', 'leiden_res_1.9000000000000001',
       'leiden_res_0.3', 'leiden_res_0.7', 'leiden_res_1.2', 'leiden_res_1.3',
       'leiden_res_1.4', 'leiden_res_1.5', 'leiden_res_1.7', 'leiden_res_1.8',
       'leiden_res_1.9', 'celltype.L1', 'celltype.L2',
       'Neurotransmitter_celltype', 'celltype.L1_ct', 'Sample_name',
       'Condition', 'Region', 'celltype.L2.raw', 'region_nt', 'celltype.L3',
       'celltype.L4', 

In [ ]:
peak_mat.obs['celltype.L1nt'] = peak_mat.obs['celltype.L1'].astype('str')
peak_mat.obs.loc[peak_mat.obs['celltype.L1'] == 'Neuron', 'celltype.L1nt'] = peak_mat.obs.loc[peak_mat.obs['celltype.L1'] == 'Neuron', 'region_nt'].astype('str').values
peak_mat.obs['celltype.L1nt'] = peak_mat.obs['celltype.L1nt'].astype('category')

In [26]:
peak_mat.write(f"output/atac0627/3REGIONS_peak.h5ads")


... storing 'sample' as categorical
... storing 'celltype.L1' as categorical
... storing 'celltype.L2' as categorical
... storing 'Neurotransmitter_celltype' as categorical
... storing 'celltype.L1_ct' as categorical
... storing 'Sample_name' as categorical
... storing 'Condition' as categorical
... storing 'Region' as categorical
... storing 'celltype.L2.raw' as categorical
... storing 'region_nt' as categorical
... storing 'celltype.L3' as categorical
... storing 'celltype.L4' as categorical


In [33]:
peak_mat.var['chr'] = peak_mat.var.index.str.split(r'[:-]').str[0]
peak_mat.var['start'] = peak_mat.var.index.str.split(r'[:-]').str[1]
peak_mat.var['end'] = peak_mat.var.index.str.split(r'[:-]').str[2]

In [34]:
peak_mat.var.to_csv('/data2st1/junyi/output/atac0627/cCRE/peak.bed', index=False, sep="\t", header=False)

In [78]:
#celltypes = adata.obs["celltype.L1.tab"].unique()
regions = ['AMY','HIP','PFC']
celltypes = ['OPC-Oligo', 'Immune','Astro-Epen','Neuron']
print(celltypes)
print(regions)

['OPC-Oligo', 'Immune', 'Astro-Epen', 'Neuron']
['AMY', 'HIP', 'PFC']


In [ ]:
# # Column to use for stratification
# stratify_column = 'sample'

# # Number of cells to sample from each group
# n_samples_per_group = 2000

# # Perform stratified sampling
# sampled_indices = (
#     adata.obs
#     .groupby(stratify_column, group_keys=False)
#     .apply(lambda x: x.sample(min(n_samples_per_group, len(x))))
#     .index
# )


/tmp/ipykernel_2089906/1039730475.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
# region = 'ALL'
# celltype = 'ALL'
# base_name = f"{region}_{celltype}"
# adata_AMY_neuron  = adata[sampled_indices, :]

# adata_AMY_neuron.obs['expriment'] = adata_AMY_neuron.obs['sample'].str[:2]

/tmp/ipykernel_2089906/4130967597.py:6: ImplicitModificationWarning:

Trying to modify attribute `.obs` of view, initializing view as actual.

